In [11]:
import os
from PyFoam.RunDictionary.FileBasis import FileBasis, FileBasisBackup

In [30]:
foam_path=os.getenv("FOAM_DIR")
path=foam_path+"/user/SandBox/pythonViz/"

In [31]:
f=FileBasisBackup(path+"pitzDaily/system/controlDict",backup=True)
f.readFile()
f.content+="\n// This file was touched by PyFoam\n"
f.writeFile()

In [37]:
f.restore()
dir(f)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'addedString',
 'backupName',
 'baseName',
 'closeFile',
 'content',
 'copyRest',
 'copyfile',
 'copytree',
 'counter',
 'diskUsage',
 'encode',
 'excludeNames',
 'execute',
 'exists',
 'fh',
 'find',
 'findFileInDir',
 'getCaseDir',
 'goMatch',
 'goTo',
 'humanReadableDuration',
 'humanReadableSize',
 'listDirectory',
 'makeTemp',
 'name',
 'openFile',
 'parse',
 'purgeFile',
 'readFile',
 'realName',
 'remove',
 'removedString',
 'restore',
 'rmtree',
 'useBinary',
 'which',
 'writeDictionaryHeader',
 'writeEncoded',
 'writeFile',
 'writeFileAs',
 'zipped']

In [36]:
os.system("tail pitzDaily/system/controlDict")



// ************************************************************************* //

// This file was touched by PyFoam

// This file was touched by PyFoam

// This file was touched by PyFoam

// This file was touched by PyFoam


0

In [38]:
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

In [39]:
cd=ParsedParameterFile(path+"pitzDaily/system/controlDict")

In [41]:
cd["writeInterval"]=int(cd["endTime"]/10)
cd.writeFile()

In [44]:
from PyFoam.Basics.FoamFileGenerator import makeString

In [45]:
makeString({"a":2,"b":[2.3,4,5]})

'a 2;\nb\n(2.3 4 5);\n'

In [49]:
from PyFoam.Infrastructure.CTestRun import CTestRun


class PlainIcoFoamCavity(CTestRun):
    def init(self):
        self.setParameters(solver="icoFoam",
                           originalCase="$FOAM_TUTORIALS/incompressible/icoFoam/cavity",
                           sizeClass="tiny")


In [53]:
PlainIcoFoamCavity().run()

Usage: ipykernel_launcher.py: [options]

ipykernel_launcher.py: error: no such option: --ip
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1427, in _process_args
    self._process_long_opt(rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1480, in _process_long_opt
    opt = self._match_long_opt(opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1465, in _match_long_opt
    return _match_abbrev(opt, self._long_opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1670, in _match_abbrev
    raise BadOptionError(s)
optparse.BadOptionError: no such option: --ip

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipyker

TypeError: object of type 'NoneType' has no len()

In [1]:
from PyFoam.Infrastructure.CTestRun import CTestRun
from PyFoam.Applications.SamplePlot import SamplePlot
from PyFoam.Basics.Data2DStatistics import Data2DStatistics


class BasicPitzDailyRun(CTestRun):
    def init(self):
        self.setParameters(solver="simpleFoam",
                           originalCase="$FOAM_TUTORIALS/incompressible/simpleFoam/pitzDaily",
                           sizeClass="small",
                           referenceData="PitzDailyData",
                           # headLength=7,
                           # tailLength=5,
                           velocityRelativeTolerance=1e-3,
                           velocityAbsoluteTolerance=1e-3,
                           minimumRunTime=100)

    def postprocess(self):
        self.execute("sample")

    def casePrepare(self):
        self.status("Modifying controlDict")
        self.controlDict()["endTime"] = 2000
        self.controlDict().writeFile()

    def postRunTestCheckConverged(self):
        self.isNotEqual(
            value=self.runInfo()["time"],
            target=self.controlDict()["endTime"],
            #            tolerance=1e-2,
            message="Reached endTime -> not converged")

    def postRunTestVelocityProfiles(self):
        comparison = self.compareSamples(
            data="sets",
            reference="referenceSet",
            fields=["U"])
        print("Relative error on sample lines")
        print(comparison.relativeError())
        self.isEqual(
            value=comparison.relativeError().max(),
            tolerance=self["velocityRelativeTolerance"],
            message="Relative error of fluid velocity")

    def postRunTestVelocityProfilesAbsolute(self):
        comparison = self.compareSamples(
            data="sets",
            reference="referenceSet",
            fields=["U"])
        print("Absolute error on sample lines")
        print(comparison.compare()["max"])
        self.isEqual(
            value=comparison.compare()["max"].max(),
            tolerance=self["velocityAbsoluteTolerance"],
            message="Absolute error of fluid velocity")

    def postRunTestVelocityProfilesOldSchool(self):
        # Just as an example how to do it detailed
        sample = SamplePlot(args=[self.caseDir,
                                  "--silent",
                                  "--dir=sets",
                                  "--reference-dir=referenceSet",
                                  "--latest-time",
                                  "--tolerant-reference-time",
                                  "--field=U",
                                  "--compare",
                                  "--metrics"])
        stat = Data2DStatistics(metrics=sample["metrics"],
                                compare=sample["compare"],
                                noStrings=True)
        relError = stat.relativeError()
        for l in relError.columns():
            for com in relError.rows():
                self.isEqual(
                    value=relError[(com, l)],
                    tolerance=self["velocityRelativeTolerance"],
                    message="Match velocty component %s on line %s" % (com, l))

    def postRunTestPressureDifference(self):
        self.isEqual(
            value=self.runInfo()["analyzed"]["deltaP"]["avg"],
            target=-5.17507,  # according to the 2.1-solver
            tolerance=0.1,
            message="Pressure difference between inlet and outlet")


if __name__ == '__main__':
    BasicPitzDailyRun().run()


Usage: ipykernel_launcher.py: [options]

ipykernel_launcher.py: error: no such option: --ip
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1427, in _process_args
    self._process_long_opt(rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1480, in _process_long_opt
    opt = self._match_long_opt(opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1465, in _match_long_opt
    return _match_abbrev(opt, self._long_opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1670, in _match_abbrev
    raise BadOptionError(s)
optparse.BadOptionError: no such option: --ip

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipyker

TypeError: object of type 'NoneType' has no len()

In [4]:
from PyFoam.Infrastructure.CTestRun import CTestRun

class PlainIcoFoamCavity(CTestRun):
    def init(self):
        self.setParameters(solver="icoFoam",
                           originalCase="$FOAM_TUTORIALS/incompressible/icoFoam/cavity/cavity/",
                           sizeClass="tiny")
PlainIcoFoamCavity().run()

Usage: ipykernel_launcher.py: [options]

ipykernel_launcher.py: error: no such option: --ip
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1427, in _process_args
    self._process_long_opt(rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1480, in _process_long_opt
    opt = self._match_long_opt(opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1465, in _match_long_opt
    return _match_abbrev(opt, self._long_opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1670, in _match_abbrev
    raise BadOptionError(s)
optparse.BadOptionError: no such option: --ip

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipyker

TypeError: object of type 'NoneType' has no len()

In [3]:
PlainIcoFoamCavity().run()

Usage: ipykernel_launcher.py: [options]

ipykernel_launcher.py: error: no such option: --ip
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1427, in _process_args
    self._process_long_opt(rargs, values)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1480, in _process_long_opt
    opt = self._match_long_opt(opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1465, in _match_long_opt
    return _match_abbrev(opt, self._long_opt)
  File "/home/zhy/anaconda3/lib/python3.7/optparse.py", line 1670, in _match_abbrev
    raise BadOptionError(s)
optparse.BadOptionError: no such option: --ip

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zhy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipyker

TypeError: object of type 'NoneType' has no len()